In [1]:
import sys
sys.path.append('..')

In [2]:
from pathlib import Path
import dietrx
from dietrx import app
from dietrx import db
from dietrx.models import *

import pandas as pd

DATA = Path('../data/version-2/')

In [3]:
def add_to_db(obj, dl):
    for i, item in enumerate(dl):
        db.session.add(obj(**item))

        if (i+1) % 10000 == 0:
            db.session.commit()
    db.session.commit()
    
    
def remove_from_db(obj):
    for i, o in enumerate(obj.query.all()):
        db.session.delete(o)
        
        if (i+1) % 10000 == 0:
            db.session.commit()
            
    db.session.commit()

### Delete previous data in DB

In [4]:
remove_from_db(Food_disease)
remove_from_db(Disease_gene)
remove_from_db(Food_gene)
remove_from_db(Food)
remove_from_db(Disease)
remove_from_db(Gene)
remove_from_db(References)
remove_from_db(Chemical)
remove_from_db(Chemical_disease)
remove_from_db(Food_chemical)

## A. Lexicons

### Food Lexicon

In [5]:
df = pd.read_csv('../data/version-2/food-lexicon.tsv', sep='\t', encoding='utf-8')
df.head()

,common-names,food-name,food-id,food-category,scientific-name,tax-id
0,alexanders; horse parsley,Alexanders,Plant ID:40962,Miscellaneous,Smyrnium olusatrum,40962.0
1,NaN,Scrophularia umbrosa,Plant ID:942083,Miscellaneous,Scrophularia umbrosa,942083.0
2,dotted blazing star,Dotted blazing star,Plant ID:344074,Miscellaneous,Liatris punctata,344074.0
3,plymouth pear,Plymouth pear,Plant ID:761867,Miscellaneous,Pyrus cordata,761867.0
4,NaN,Rhododendron kaempferi,Plant ID:49166,Miscellaneous,Rhododendron kaempferi,49166.0


In [6]:
# Replace NaN with None
df = df.where((pd.notnull(df)), None)

# Rename columns to match db schema.
df.rename(columns={
    'common-names': 'common_names',
    'food-name': 'display_name',
    'food-id': 'food_id',
    'food-category': 'food_category',
    'scientific-name': 'scientific_name',
    'tax-id':'tax_id',
}, inplace=True)

data_list= list(df.T.to_dict().values())

In [7]:
add_to_db(Food, data_list)

### Disease Lexicon

In [8]:
df = pd.read_csv('../data/version-2/disease-lexion.tsv', sep='\t', encoding='utf-8', index_col=0)
df.reset_index(inplace=True)
df.head()

,disease-id,disease-name,other-ids,parent-ids,disease-treenum,disease-synonyms,disease-category
0,OMIM:616890,SPLIT-FOOT MALFORMATION WITH MESOAXIAL POLYDAC...,NaN,MESH:D017689|MESH:D017880,C05.660.585.600/616890|C05.660.585/616890|C16....,SFMMP,Congenital abnormality|Musculoskeletal disease
1,MESH:D008924,Mite Infestations,DO:DOID:7894,MESH:D004478,C03.858.211.480,"Acariasis|Infestation, Mite|Infestations, Mite...",Parasitic disease
2,MESH:D012600,Scoliosis,DO:DOID:0060249,MESH:D013121,C05.116.900.800.875,Scolioses,Musculoskeletal disease
3,MESH:D002291,"Carcinoma, Papillary",DO:DOID:3113,MESH:D002277|MESH:D018307,C04.557.470.200.360|C04.557.470.700.360,"Carcinomas, Papillary|Papillary Carcinoma|Papi...",Cancer
4,MESH:C567254,Diamond-Blackfan Anemia 7,OMIM:612562,MESH:D029503,C15.378.071.085.080.090/C567254|C15.378.071.75...,DBA7,Blood disease|Genetic disease (inborn)


In [9]:
df = df.where((pd.notnull(df)), None)

# Rename to match schema
df.rename(columns={
    'disease-id': 'disease_id',
    'disease-name': 'disease_name',
    'disease-category': 'disease_category',
    'disease-synonyms': 'disease_synonyms'
}, inplace=True)

# Keep only selected columns.
df = df[['disease_id', 'disease_name', 'disease_category']]

data_list= list(df.T.to_dict().values())

In [10]:
add_to_db(Disease, data_list)

### Gene Lexicon

In [11]:
df = pd.read_csv('../data/version-2/gene-disease.tsv', sep='\t', encoding='utf-8')[['gene-id']]
df.drop_duplicates(subset=['gene-id'], inplace=True)
df.head()

,gene-id
0,MAP3K20
1,DLL3
2,NOTCH1
3,RPL11
4,OFC12


In [12]:
df.rename(columns={
    'gene-id': 'gene_id',
}, inplace=True)


data_list= list(df.T.to_dict().values())

In [13]:
add_to_db(Gene, data_list)

### References

In [14]:
df = pd.read_csv('../data/version-2/publication-records.tsv', sep='\t', encoding='utf-8')

df.head()

,pmid,date,authors,title,publication-type,journal-name,journal-name-abbrv
0,7398283,1980 Jun,Kauppinen K|Kousa M|Reunala T,Aromatic plants--a cause of severe attacks of ...,Journal Article,Contact dermatitis,Contact Dermatitis
1,24268374,2014 May,Vilhena-Churchill N|Goldstein AL,Child maltreatment and marijuana problems in y...,Journal Article,Child abuse & neglect,Child Abuse Negl
2,24273069,2014 Feb,Wang P|Su C|Li R|Wang H|Ren Y|Sun H|Yang J|Sun...,Mechanisms and effects of curcumin on spatial ...,Journal Article,Journal of neuroscience research,J Neurosci Res
3,24274004,2013 Jul-Sep,Arghir OC|Danteş E|Stoicescu R|Baicu I|Halichi...,Parental environmental tobacco smoking and the...,Journal Article,"Pneumologia (Bucharest, Romania)",Pneumologia
4,24274317,2013 Nov 25,Lee DH|Seo ES|Hong JT|Lee GT|You YK|Lee KK|Jo ...,The efficacy and safety of a proposed herbal m...,Journal Article|Randomized Controlled Trial,BMC complementary and alternative medicine,BMC Complement Altern Med


In [15]:
df = df.where((pd.notnull(df)), None)

df.rename(columns={
    'authors': 'authors',
    'journal-name': 'journal_name',
    'journal-name-abbrv': 'journal_name_abbr',
    'publication-type':'publication_type',
},inplace=True)

data_list= list(df.T.to_dict().values())

In [16]:
add_to_db(References, data_list)

## B. Associations

### Food-disease

In [17]:
df = pd.read_csv('../data/version-2/food-disease.tsv', sep='\t', encoding='utf-8')
df = df.where((pd.notnull(df)), None)
df['association'] = df['association'].apply(str.lower)

df.head()

,pmid,food-id,disease-id,association
0,7398283,Plant ID:4045,MESH:D004487,negative
1,7398283,Plant ID:4045,MESH:D014581,negative
2,7398283,Plant ID:4043,MESH:D004487,negative
3,7398283,Plant ID:4043,MESH:D014581,negative
4,24268374,Plant ID:3483,MESH:D021081,negative


In [18]:
df.rename(columns={
    'PMID': 'pmid',
    'food-id': 'food_id',
    'disease-id': 'disease_id',
    'association': 'association',
}, inplace=True)

data_list= list(df.T.to_dict().values())

In [19]:
add_to_db(Food_disease, data_list)

### Gene-disease

In [20]:
df = pd.read_csv('../data/version-2/gene-disease.tsv', sep='\t', encoding='utf-8')

# Merge duplicate associations
df['source'] = df['source'].map(lambda s: s.split(', '))
df = df.groupby(['disease-id', 'gene-id']).agg(lambda s: set([g for glst in s for g in glst])).reset_index()
df['source'] = df['source'].map(lambda s: ', '.join(s))

df.rename(columns={
    'gene-id': 'gene_id',
    'disease-id': 'disease_id',
    'source': 'reference',
}, inplace=True)
df.head()

,disease_id,gene_id,reference
0,MESH:C000591739,CYP19A1,ORPHANET
1,MESH:C000596385,CNNM4,UNIPROT
2,MESH:C000598645,MYOT,"CTD_human, UNIPROT, ORPHANET"
3,MESH:C000600608,CHST14,"CTD_human, UNIPROT, ORPHANET"
4,MESH:C000600608,DSE,ORPHANET


In [21]:
data_list= list(df.T.to_dict().values())

In [22]:
add_to_db(Disease_gene, data_list)

###  Food-Gene

In [23]:
# Find food-gene associations
fd = pd.read_csv('../data/version-2/food-disease.tsv', sep='\t', encoding='utf-8')
gd =pd.read_csv('../data/version-2/gene-disease.tsv', sep='\t', encoding='utf-8')
food_gene = fd.merge(gd, on='disease-id').drop(['pmid', 'association', 'source'], axis=1)

In [24]:
food_gene.rename(columns={
    'food-id': 'food_id',
    'gene-id': 'gene_id',
},inplace=True)

del food_gene['disease-id']
food_gene.disease_categories = ''

food_gene.drop_duplicates(inplace=True)

data_list= list(food_gene.T.to_dict().values())

In [25]:
add_to_db(Food_gene, data_list)

In [26]:
app.elasticsearch.indices.delete(index='food', ignore=[400, 404])
app.elasticsearch.indices.delete(index='disease', ignore=[400, 404])
app.elasticsearch.indices.delete(index='gene', ignore=[400, 404])


Food.reindex("food_id")
Disease.reindex("disease_id")
Gene.reindex("gene_id")

### Chemicals

In [126]:
cd = pd.read_csv('../data/version-2/chemical-disease.tsv', sep='\t', encoding='utf-8')
fc = pd.read_csv('../data/version-2/food-chemical.tsv', sep='\t', encoding='utf-8')


cd['pubchem-id'] = cd['pubchem-id'].astype(int)
fc['pubchem-id'] = fc['pubchem-id'].astype(int)

cd.rename(columns={
    'pubchem-id': 'pubchem_id',
    'disease-id': 'disease_id',
}, inplace=True)


fc.rename(columns={
    'pubchem-id': 'pubchem_id',
    'food-id': 'food_id',
    'references': 'references',
}, inplace=True)

fc.drop_duplicates(inplace=True)

v = fc.groupby(['food_id', 'pubchem_id']).references.apply(lambda x: x.tolist())

def combine(x):
    x = x.tolist()[2:8]
    return '; '.join(list(filter(None, x)))


df = pd.DataFrame(v.tolist(), index=v.index)\
       .reset_index()
df['references'] = df.apply(lambda x:combine(x), axis=1)
df = df[['food_id', 'pubchem_id', 'references']]

fc = df

In [127]:
cd.head()

,pubchem_id,disease_id
0,5280445,MESH:D007249
1,5280445,MESH:D009202
2,5280445,MESH:D002386
3,5280445,MESH:D010051
4,5280445,MESH:D007674


In [128]:
fc.head()

,food_id,pubchem_id,references
0,MeatEgg ID:9913,247,USDA; U
1,MeatEgg ID:9913,5810,USDA; U
2,MeatEgg ID:9913,6322,DTU; USDA; U
3,MeatEgg ID:9913,152059,USDA; U
4,MeatEgg ID:9913,171548,DTU


In [129]:
data_list= list(cd.T.to_dict().values())
add_to_db(Chemical_disease, data_list)

In [130]:
data_list= list(fc.T.to_dict().values())
add_to_db(Food_chemical, data_list)

### Generating chemical pubchems and adding to DB

In [135]:
pubchem_ids = list(set(fc['pubchem_id'].tolist() + cd['pubchem_id'].tolist()))

for i, item in enumerate(pubchem_ids):
    db.session.add(Chemical(pubchem_id = item))

    if (i+1) % 10000 == 0:
        db.session.commit()
db.session.commit()